## Variables Set

In [11]:
PROJECT_ID = 'jchavezar-demo' # @param {type:"string"}
REGION = 'us-central1' # @param {type:"string"}
MODEL_NAME = 'segformer-b0-finetuned-segments-sockcop-march-8'
STAGING_URI = 'gs://vtx-staging/image_segmentation_road/gpu' # @param {type:"string"}
TRAIN_IMAGE_URI = 'gcr.io/jchavezar-demo/pytorch-gpu:v2' # @param {type:"string"}
#PREDICTION_IMAGE_URI = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest' # @param {type:"string"}

In [6]:
import sys
sys.path.append('../../../../tkn')

from sec import TOKEN

In [7]:
!rm -fr source
!mkdir source

## Training and Image Files

In [12]:
%%writefile source/train.py
#%%
import json
import torch
from torch import nn
import evaluate
import multiprocessing
import argparse
from transformers import Trainer
from datasets import load_dataset
from huggingface_hub import hf_hub_download
from torchvision.transforms import ColorJitter
from transformers import (
    SegformerFeatureExtractor,
)
from transformers import TrainingArguments
from transformers import SegformerForSemanticSegmentation

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

parser = argparse.ArgumentParser()
parser.add_argument('--token', type=str,
                    help='github token')
parser.add_argument('--model_name', type=str,
                    help='github model name')
args = parser.parse_args()

feature_extractor = SegformerFeatureExtractor()
hf_username = "segments-sockcop"
hf_dataset_identifier = "segments/sidewalk-semantic"
pretrained_model_name = "nvidia/mit-b0"
hub_model_id = args.model_name

def extract_preprocess():
    print('[INFO] ------ Data Extraction and Preprocess')
    ds = load_dataset(hf_dataset_identifier)

    ds = ds.shuffle(seed=1)
    ds = ds["train"].train_test_split(test_size=0.2)
    train_ds = ds["train"]
    test_ds = ds["test"]

    filename = "id2label.json"
    id2label = json.load(open(hf_hub_download(repo_id=hf_dataset_identifier, filename=filename, repo_type="dataset"), "r"))
    id2label = {int(k): v for k, v in id2label.items()}
    label2id = {v: k for k, v in id2label.items()}
    num_labels = len(id2label)
    print("Id2label:", id2label)

    jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1) 

    def train_transforms(example_batch):
        print('[INFO] ------ Training Transform Process')
        images = [jitter(x) for x in example_batch['pixel_values']]
        labels = [x for x in example_batch['label']]
        inputs = feature_extractor(images, labels)
        return inputs


    def val_transforms(example_batch):
        print('[INFO] ------ Validation Transform Process')
        images = [x for x in example_batch['pixel_values']]
        labels = [x for x in example_batch['label']]
        inputs = feature_extractor(images, labels)
        return inputs

    train_ds.set_transform(train_transforms)
    test_ds.set_transform(val_transforms)

    # Set transforms
    return train_ds, test_ds, id2label, label2id

def trainer_build(train_ds, test_ds, id2label, label2id):
    print('[INFO] ------ Training Build Process')
    epochs = 50
    lr = 0.00006
    batch_size = 1 
    model = SegformerForSemanticSegmentation.from_pretrained(
        pretrained_model_name,
        id2label=id2label,
        label2id=label2id
    )

    training_args = TrainingArguments(
        "segformer-b0-finetuned-segments-sidewalk-outputs",
        learning_rate=lr,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        save_total_limit=3,
        evaluation_strategy="steps",
        save_strategy="steps",
        save_steps=20,
        eval_steps=20,
        logging_steps=1,
        eval_accumulation_steps=5,
        load_best_model_at_end=True,
        push_to_hub=True,
        hub_model_id=hub_model_id,
        hub_strategy="end",
        hub_token=args.token
    )

    metric = evaluate.load("mean_iou")

    def compute_metrics(eval_pred):
        with torch.no_grad():
            logits, labels = eval_pred
            logits_tensor = torch.from_numpy(logits)
            # scale the logits to the size of the label
            logits_tensor = nn.functional.interpolate(
                logits_tensor,
                size=labels.shape[-2:],
                mode="bilinear",
                align_corners=False,
            ).argmax(dim=1)

            pred_labels = logits_tensor.detach().cpu().numpy()
            metrics = metric._compute(
                    predictions=pred_labels,
                    references=labels,
                    num_labels=len(id2label),
                    ignore_index=0,
                    reduce_labels=feature_extractor.reduce_labels,
                )
            
            # add per category metrics as individual key-value pairs
            per_category_accuracy = metrics.pop("per_category_accuracy").tolist()
            per_category_iou = metrics.pop("per_category_iou").tolist()

            metrics.update({f"accuracy_{id2label[i]}": v for i, v in enumerate(per_category_accuracy)})
            metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})

            return metrics

    trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
    )
    
    trainer.train()

    print("[INFO] ------ Pushing Model to HuggingHub")
    kwargs = {
    "tags": ["vision", "image-segmentation"],
    "finetuned_from": pretrained_model_name,
    "dataset": hf_dataset_identifier,
    }
    feature_extractor.push_to_hub(hub_model_id)
    trainer.push_to_hub(**kwargs)



def main():
    train_ds, test_ds, id2label, label2id = extract_preprocess()
    print(train_ds)
    trainer = trainer_build(train_ds, test_ds, id2label, label2id)

if __name__ == "__main__":
    main()
# %%

Overwriting source/train.py


In [13]:
%%writefile source/Dockerfile

FROM nvcr.io/nvidia/pytorch:21.02-py3

COPY . .

RUN curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash
RUN apt-get update -y
RUN apt-get install git-lfs
RUN pip install jupyter ipykernel 
RUN pip install transformers datasets segments-ai evaluate git-lfs tqdm==4.59

ENTRYPOINT ["python", "train.py"]

Overwriting source/Dockerfile


## Push To Google Cloud Repository

In [14]:
!gcloud builds submit -t $TRAIN_IMAGE_URI source/.

Creating temporary tarball archive of 2 file(s) totalling 5.6 KiB before compression.
Uploading tarball of [source/.] to [gs://jchavezar-demo_cloudbuild/source/1678254969.236111-211bd99c4def40b090a4392742f46345.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/jchavezar-demo/locations/global/builds/87169393-7432-404c-8de0-84e4d769d519].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/87169393-7432-404c-8de0-84e4d769d519?project=569083142710 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "87169393-7432-404c-8de0-84e4d769d519"

FETCHSOURCE
Fetching storage object: gs://jchavezar-demo_cloudbuild/source/1678254969.236111-211bd99c4def40b090a4392742f46345.tgz#1678254969463133
Copying gs://jchavezar-demo_cloudbuild/source/1678254969.236111-211bd99c4def40b090a4392742f46345.tgz#1678254969463133...
/ [1 files][  2.1 KiB/  2.1 KiB]                                                
Operation completed over 1 ob

## Train Job (CustomJob)

In [18]:
from google.cloud import aiplatform as aip

aip.init(
    project=PROJECT_ID,
    location=REGION)


worker_pool_specs = [
    {
        'machine_spec' : {
            'machine_type': 'a2-highgpu-2g',
            'accelerator_type': 'NVIDIA_TESLA_A100',
            'accelerator_count': 2
        },
        'replica_count': 1,
        
        'container_spec': {
            'image_uri': TRAIN_IMAGE_URI,
            'args': [
                '--token='+TOKEN,
                '--model_name='+MODEL_NAME,
            ]
        }
    }
]

job = aip.CustomJob(
    display_name = 'image_seg_custom_py',
    worker_pool_specs = worker_pool_specs,
    staging_bucket = STAGING_URI
)

model = job.run()

Creating CustomJob
CustomJob created. Resource name: projects/569083142710/locations/us-central1/customJobs/4695452998984269824
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/569083142710/locations/us-central1/customJobs/4695452998984269824')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/4695452998984269824?project=569083142710
CustomJob projects/569083142710/locations/us-central1/customJobs/4695452998984269824 current state:
JobState.JOB_STATE_QUEUED
CustomJob projects/569083142710/locations/us-central1/customJobs/4695452998984269824 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/569083142710/locations/us-central1/customJobs/4695452998984269824 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/569083142710/locations/us-central1/customJobs/4695452998984269824 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/569083142710/locations/us-central1/customJobs/46954529

RuntimeError: Job failed with:
code: 1
message: "CANCELED"
